## Проектная работа "SQL"
**Выполнил:** Киселев Дмитрий, da_58

**Задача:**

Проанализировать базу данных сервиса для чтения книг по подписке. Имеются несколько разделов с информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.


### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [1]:
# импортируем библиотеки
import pandas as pd
from yaml import load, FullLoader
from sqlalchemy import create_engine


In [2]:
# загружаем параметры подключения к БД из локального файла с настройками

db_config=load(open(r'C:\Users\kslvd\pyn_projects\yandex_projects\final_project_SQL\DB_config.yaml'), Loader=FullLoader)

In [3]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [4]:
# выведем первые 10 строк таблицы books
query="""
SELECT *
FROM books
LIMIT 10
"""

In [5]:
pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


In [6]:
# выведем первые 10 строк таблицы ratings
query2="""
SELECT *
FROM ratings
LIMIT 10
"""

In [7]:
pd.io.sql.read_sql(query2, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4


In [8]:
# выведем первые 10 строк таблицы reviews
query3="""
SELECT *
FROM reviews
LIMIT 10
"""

In [9]:
pd.io.sql.read_sql(query3, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With pe...
7,8,4,abbottjames,Within enough mother. There at system full rec...
8,9,5,npowers,Thank now focus realize economy focus fly. Ite...
9,10,5,staylor,Game push lot reduce where remember. Including...


In [10]:
# Посчитаем, сколько книг вышло после 1 января 2000 года
books_after_date="""
WITH this AS
    (SELECT publication_date,
            COUNT(DISTINCT book_id) AS books_cnt
    FROM books
    WHERE publication_date >= '2000-01-01'
    GROUP BY publication_date)
SELECT SUM (books_cnt)
FROM this
"""

In [11]:
pd.io.sql.read_sql(books_after_date, con = engine)


,sum
0,821.0


Всего у нашего сервиса имеются записи о 821 книге вышедшей после 1 янв 2000 года.

In [12]:
# Для каждой книги посчитаем количество обзоров и среднюю оценку, выведем первые 20 строк
reviews_ratings="""
WITH rats AS
    (SELECT book_id,
            AVG(rating) AS avg_rating
    FROM ratings
    GROUP BY book_id),
revs AS
    (SELECT book_id,
            COUNT(DISTINCT review_id) AS reviews_cnt
    FROM reviews
    GROUP BY book_id)
SELECT books.book_id,
       books.title,
       revs.reviews_cnt,
       rats.avg_rating
FROM books
LEFT JOIN rats ON rats.book_id = books.book_id
LEFT JOIN revs ON revs.book_id = books.book_id
ORDER BY reviews_cnt DESC
LIMIT 20
"""

In [13]:
pd.io.sql.read_sql(reviews_ratings, con = engine)

,book_id,title,reviews_cnt,avg_rating
0,808,The Natural Way to Draw,NaN,3.000000
1,387,Leonardo's Notebooks,NaN,4.000000
2,221,Essential Tales and Poems,NaN,4.000000
3,83,Anne Rice's The Vampire Lestat: A Graphic Novel,NaN,3.666667
4,672,The Cat in the Hat and Other Dr. Seuss Favorites,NaN,5.000000
5,191,Disney's Beauty and the Beast (A Little Golden...,NaN,4.000000
6,948,Twilight (Twilight #1),7.0,3.662500
7,963,Water for Elephants,6.0,3.977273
8,854,The Road,6.0,3.772727
9,207,Eat Pray Love,6.0,3.395833


Имеется 6 книг, на которые нет отзывов от пользователей сервиса. Самая популярная на составление отзыва книга Сумерки не обладает высоким рейтингом, даже наоборот ее средний рейтинг хуже других популярных книг. В целом в дальнейшем анализе хочется посмотреть на связь количества отзывов и рейтинга, это поможет определить поведение и интересы пользователей.

In [14]:
# Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так из анализа удалятся брошюры 
top_publisher="""
SELECT books.publisher_id,
       publishers.publisher,
       COUNT(DISTINCT books.book_id) AS books_cnt
FROM books
LEFT JOIN publishers ON publishers.publisher_id = books.publisher_id
WHERE books.num_pages > 50
GROUP BY books.publisher_id, publishers.publisher
ORDER BY books_cnt DESC
LIMIT 5
"""

In [15]:
pd.io.sql.read_sql(top_publisher, con = engine)

,publisher_id,publisher,books_cnt
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,33,Ballantine Books,19


Для сервиса самый плодотворный издатель - Penguin Books, с большим отрывом от следующего издателя у нас 42 книги (без учета брошюр).

In [16]:
# Определим автора с самой высокой средней оценкой книг — учтем только книги с 50 и более оценками
top_author="""
WITH top AS
    (SELECT book_id,
            AVG (rating) AS avg_rating,
            COUNT(DISTINCT rating_id) AS marks_cnt
    FROM ratings
    GROUP BY book_id
    HAVING COUNT(DISTINCT rating_id) >= 50
    ORDER BY avg_rating DESC)
    
SELECT books.author_id,
       AVG(top.avg_rating) AS author_rating,
       authors.author
FROM top
LEFT JOIN books ON top.book_id = books.book_id
LEFT JOIN authors ON authors.author_id = books.author_id
GROUP BY books.author_id, authors.author
ORDER BY author_rating DESC
LIMIT 3      
"""

In [17]:
pd.io.sql.read_sql(top_author, con = engine)

,author_id,author_rating,author
0,236,4.283844,J.K. Rowling/Mary GrandPré
1,402,4.264151,Markus Zusak/Cao Xuân Việt Khương
2,240,4.258446,J.R.R. Tolkien


Самым высоко оцениваемым автором книг по мнению наших пользователей является Джоан Роулинг с иллюстрациями Мэри Грандпре. Очевидно это влияние серии Гарри Поттера, слега отстают от первого места Markus Zusak/Cao Xuân Việt Khương и J.R.R. Tolkien

In [18]:
# Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок
active_users="""
WITH top_raters AS
    (SELECT username,
           COUNT(DISTINCT review_id) AS reviews_cnt
    FROM reviews
    WHERE username IN 
        (SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(DISTINCT rating_id) > 50)
    GROUP BY username)
    
SELECT AVG (reviews_cnt)
FROM top_raters
"""

In [19]:
pd.io.sql.read_sql(active_users, con = engine)

,avg
0,24.333333


Имеются самые активные пользователи которые поставили более 50 оценок, в среднем они также составляют 24 отзыва, но их не так много.

**Вывод:** 
Сервис имеет достаточно много книг, пользователи вполне активны - ставят оценки и создают отзывы. Самый многочисленный издатель Penguin Books, но есть и множество других. Читатели любят Джоан Роулинг, и самые активные из них (кто поставил более 50 оценок) даже способны составить 24 отзыва на разные книги. Получается что кроме хороших книг по подписке у сервиса имеются активные пользователи, которые могут поднять общую читаемость и популярность сервиса по подписке. Возможно следует получше изучить интересы аудитории и заняться ассортиментом книг.